# World Bank

Steps: 
1. Fetch World Bank data for the indicator 'IT.NET.USER.ZS' (Internet users).
2. Normalize JSON response into a clean pandas DataFrame.
3. Filter to only valid countries of interest.
4. Keep the most recent year with a non-NA value for each country.
5. Save the processed data to a CSV file.

## ETL Pipeline

In [ ]:
import requests
import pandas as pd
import os

os.makedirs('./filtered_data', exist_ok=True)

In [10]:
# set of countries to keep in the final dataset
valid_countries = {
    'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda',
    'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The',
    'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
    'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam',
    'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
    'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.',
    'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
    'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.',
    'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
    'Fiji', 'Finland', 'France', 'Gabon', 'Gambia, The', 'Georgia', 'Germany', 'Ghana',
    'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti',
    'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Rep.',
    'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan',
    'Kenya', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.', 'Kuwait',
    'Kyrgyz Republic', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya',
    'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali',
    'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius', 'Mexico',
    'Micronesia, Fed. Sts.', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco',
    'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Zealand',
    'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan',
    'Palau', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland',
    'Portugal', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda', 'Samoa',
    'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia',
    'Seychelles', 'Sierra Leone', 'Singapore', 'Slovak Republic', 'Slovenia',
    'Solomon Islands', 'Somalia, Fed. Rep.', 'South Africa', 'South Sudan', 'Spain',
    'Sri Lanka', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines',
    'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Tajikistan',
    'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago',
    'Tunisia', 'Turkiye', 'Turkmenistan', 'Tuvalu', 'Uganda', 'Ukraine',
    'United Arab Emirates', 'United Kingdom', 'United States', 'Uruguay',
    'Uzbekistan', 'Vanuatu', 'Venezuela, RB', 'Viet Nam', 'Yemen, Rep.', 'Zambia',
    'Zimbabwe'
}

In [55]:
"""

    Fetch data from World Bank API for all pages in the range 2000–2023.

    Returns a pandas DataFrame of raw records.

"""

def fetch_worldbank_data(base_url, headers, per_page=1000, timeout=60):
    all_records = []
    page = 1
    
    while True:
        url = f"{base_url}?format=json&per_page={per_page}&date=2000:2023&page={page}"
        r = requests.get(url, headers=headers, timeout=timeout)
        r.raise_for_status()
        data = r.json()

        # Stop if response is empty or malformed
        if not data or len(data) < 2:
            break
        meta, records = data[0], data[1]
        all_records.extend(records)

        # Stop if we reached the last page
        if meta["page"] >= meta["pages"]:
            break
        page += 1

    return pd.DataFrame(all_records)

### Normalize The Data

In [36]:
"""

    Normalize World Bank raw DataFrame:
    - Extract country names and indicator names
    - Rename columns for clarity
    - Convert year and value to numeric types
    
 """
def normalize_worldbank_df(df):
    df_clean = df.copy()
    df_clean["country"] = df_clean["country"].apply(lambda x: x["value"] if isinstance(x, dict) else None)
    df_clean["indicator_name"] = df_clean["indicator"].apply(lambda x: x["value"] if isinstance(x, dict) else INDICATOR)
    df_clean = df_clean.rename(columns={
        "countryiso3code": "iso3",
        "date": "year",
        "value": "value"
    })
    df_clean = df_clean[["country", "iso3", "year", "value", "indicator_name"]]
    df_clean["year"] = pd.to_numeric(df_clean["year"], errors="coerce").astype("Int64")
    df_clean["value"] = pd.to_numeric(df_clean["value"], errors="coerce")
    
    indicator_col_name = df_clean["indicator_name"].iloc[0] 
    df_clean = df_clean.rename(columns={"value": indicator_col_name})
    df_clean = df_clean.drop(columns=["indicator_name"])

    
    return df_clean

### Filter to Valid Countries

In [37]:
"""

    Keep only rows where country is in the valid_countries set

"""
def filter_valid_countries(df, valid_countries):
    return df[df["country"].isin(valid_countries)].reset_index(drop=True)

## Fetching Internet Users

In [ ]:
INDICATOR_net = "IT.NET.USER.ZS" # Internet users (% of population)
BASE_URL_net = f"https://api.worldbank.org/v2/country/all/indicator/{INDICATOR_net}"
HEADERS_net = {
    "User-Agent": "Python-requests/2.0 (isabelabarton@gmail.com)",
    "From": "isabelabarton@gmail.com"
}
df_net_raw = fetch_worldbank_data(BASE_URL_net, HEADERS_net)
df_net_norm = normalize_worldbank_df(df_net_raw)
df_net_countries = filter_valid_countries(df_net_norm, valid_countries)
df_net_countries.to_csv('./filtered_data/worldbank_net_all_years.csv', index=False)

## Fetching GDP Per Capita

In [ ]:
INDICATOR_gdp = "NY.GDP.PCAP.CD"
BASE_URL_gdp = f"https://api.worldbank.org/v2/country/all/indicator/{INDICATOR_gdp}"
HEADERS_gdp = {
    "User-Agent": "Python-requests/2.0 (isabelabarton@gmail.com)",
    "From": "isabelabarton@gmail.com"
}
df_gdp_raw = fetch_worldbank_data(BASE_URL_gdp, HEADERS_gdp)
df_gdp_norm = normalize_worldbank_df(df_gdp_raw)
df_gdp_countries = filter_valid_countries(df_gdp_norm, valid_countries)
df_gdp_countries.to_csv('./filtered_data/worldbank_gdp_per_capita.csv', index=False)

## Fetching Population

In [ ]:
INDICATOR_pop = "SP.POP.TOTL"
BASE_URL_pop = f"https://api.worldbank.org/v2/country/all/indicator/{INDICATOR_pop}"
HEADERS_pop = {
    "User-Agent": "Python-requests/2.0 (isabelabarton@gmail.com)",
    "From": "isabelabarton@gmail.com"
}
df_pop_raw = fetch_worldbank_data(BASE_URL_pop, HEADERS_pop, timeout=180)
df_pop_norm = normalize_worldbank_df(df_pop_raw)
df_pop_countries = filter_valid_countries(df_pop_norm, valid_countries)
df_pop_countries.to_csv('./filtered_data/worldbank_population.csv', index=False)


## Fetching Urbanization

In [60]:
INDICATOR_urban = "SP.URB.TOTL.IN.ZS"
BASE_URL_urban = f"https://api.worldbank.org/v2/country/all/indicator/{INDICATOR_urban}"
HEADERS_urban = {
    "User-Agent": "Python-requests/2.0 (isabelabarton@gmail.com)",
    "From": "isabelabarton@gmail.com"
}
df_urban_raw = fetch_worldbank_data(BASE_URL_urban, HEADERS_urban, timeout=180)
df_urban_norm = normalize_worldbank_df(df_urban_raw)
df_urban_countries = filter_valid_countries(df_urban_norm, valid_countries)
df_urban_countries.to_csv('./filtered_data/worldbank_urbanization.csv', index=False)
